<a href="https://colab.research.google.com/github/millygupta/Speech-recognition/blob/main/Assignment_doris2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the neccessary libraries
# Data processing
import pandas as pd
import numpy as np
import scipy.stats

# Visualization
import seaborn as sns

# Similarity
from sklearn.metrics.pairwise import cosine_similarity #used for checking the similarity between users

In [ ]:
#reading the data--

# order_products has order_id and product_id
# products has product_id
# orders has user_id  and order_id

import pandas as pd

order_products1 = '/content/order_products.csv'
order_products = pd.read_csv(order_products1)
orders1= "/content/orders.csv"
orders = pd.read_csv(orders1)
product1= "/content/products.csv"
product = pd.read_csv(product1)

# New Section

In [ ]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [ ]:
order_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


In [ ]:
product.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [ ]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [ ]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


In [ ]:
# Number of products
print('The order_products dataset has', order_products['product_id'].nunique(), 'unique products')

# Number of orders
print('The order_products dataset has', order_products['order_id'].nunique(), 'unique orders')

# Number of users
print('The orders dataset has', orders['user_id'].nunique(), 'unique users')

The order_products dataset has 49677 unique products
The order_products dataset has 3214874 unique orders
The orders dataset has 206209 unique users


In [ ]:
# merging --
# df == merge on (orders & order_products) with order_id as the primary key --- it will give user_id, order_id, product_id
df = pd.merge(orders, order_products, on='order_id', how='inner')

# Take a look at the data
df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [ ]:
# again merging ---
# df2 == merge on (df & products)  with prouct id  as the primary key
df2 = pd.merge(df, product, on='product_id', how='inner')

# Take a look at the data
df2.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7


In [ ]:
df2['product_name'].value_counts() #checking the unique values and their value_counts

Banana                                          472565
Bag of Organic Bananas                          379450
Organic Strawberries                            264683
Organic Baby Spinach                            241921
Organic Hass Avocado                            213584
                                                 ...  
Organic Finely Shredded Sharp Cheddar Cheese         1
Vegan Coconut Milk Caramels With Sea Salt            1
Pappardelle Nests Pasta                              1
Peas Pulav Basmati Rice With Green Peas              1
Hot Oatmeal Multigrain Raisin                        1
Name: product_name, Length: 49677, dtype: int64

In [ ]:
df2['product_name'].nunique() #checking the unique values and their value_counts

49677

In [ ]:
df2

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32434484,1320836,202557,prior,17,2,15,1.0,43553,2,1,Orange Energy Shots,64,7
32434485,31526,202557,prior,18,5,11,3.0,43553,2,1,Orange Energy Shots,64,7
32434486,758936,203436,prior,1,2,7,NaN,42338,4,0,"Zucchini Chips, Pesto",50,19
32434487,2745165,203436,prior,2,3,5,15.0,42338,16,1,"Zucchini Chips, Pesto",50,19


In [ ]:
df2['order_number'].value_counts() #checking the unique values and their value_counts

1     2078068
3     2050731
2     2048332
4     1820298
5     1628411
       ...   
95      14528
96      13746
97      13367
98      12858
99      12436
Name: order_number, Length: 99, dtype: int64

In [ ]:
## Now we need to filter out some data based on order numbers, will only take rows with product orders greater than 500
# Aggregate by products
agg_orders = df2.groupby('product_name').agg(mean_orders = ('order_number', 'mean'),
                                                number_of_orders = ('order_number', 'count')).reset_index()

# Keep the movies with over 500 order number
agg_orders_GT500 = agg_orders[agg_orders['number_of_orders']>500]
agg_orders_GT500.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8280 entries, 0 to 49661
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_name      8280 non-null   object 
 1   mean_orders       8280 non-null   float64
 2   number_of_orders  8280 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 258.8+ KB


In [ ]:
#Let's check what the most popular products and their order numbers are.
agg_orders_GT500.sort_values(by='number_of_orders', ascending=False).head()

,product_name,mean_orders,number_of_orders
3676,Banana,17.233572,472565
3471,Bag of Organic Bananas,20.463315,379450
31920,Organic Strawberries,17.618857,264683
28840,Organic Baby Spinach,17.531070,241921
30297,Organic Hass Avocado,19.420443,213584


In [ ]:
agg_orders_GT500

,product_name,mean_orders,number_of_orders
0,#2 Coffee Filters,19.308880,777
13,0% Fat Blueberry Greek Yogurt,12.345936,529
14,0% Fat Free Organic Milk,14.764460,3821
17,0% Fat Organic Greek Vanilla Yogurt,19.005161,1744
20,0% Fat Superfruits Greek Yogurt,12.051893,713
...,...,...,...
49646,of Hanover 100 Calorie Pretzels Mini,17.727158,3163
49651,smartwater® Electrolyte Enhanced Water,15.586525,4883
49652,vitaminwater® XXX Acai Blueberry Pomegranate,16.365197,1747
49657,with Crispy Almonds Cereal,21.385450,4330


In [ ]:
#keeping only the dataset which has orders more than 500
# Merge data
df3 = pd.merge(df2, agg_orders_GT500[['product_name']], on='product_name', how='inner')
df3.info()

In [ ]:
df3

In [ ]:
#Now we only need the user_id, product_name
# Creating user-product dataset
df4 = df3.pivot_table(index='user_id', columns='product_name', values='order_number')
df4.head() #this will mean that how many times a user has ordered a specific product

In [ ]:
# Normalize user-product matrix
df5 = df4.subtract(df4.mean(axis=1), axis = 'rows')
df5.head()

In [ ]:
# User similarity matrix using Pearson correlation
user_similarity = df5.T.corr()
user_similarity.head()

In [ ]:
# Pick a user ID=7  #that is the target user, we are going to recommend to him/her
picked_userid = 7

# Remove picked user ID from the candidate list
user_similarity.drop(index=picked_userid, inplace=True)

# Take a look at the data
user_similarity.head()
#if the value comes out to be negative then it means the correlation is not there and its opposite, if it comes out to be positive then that means it is positively correlated.

In [ ]:
#Now going to set a threshhold value so that we can get the users who are similar to used id 7
# Number of similar users
n= 8

# User similarity threashold
user_similarity_threshold = 0.2

# Get top n similar users
similar_users = user_similarity[user_similarity[picked_userid]>user_similarity_threshold][picked_userid].sort_values(ascending=False)[:n]

# Print out top n similar users
print(f'The similar users for user {picked_userid} are', similar_users)

In [ ]:
# removing everything jut keeping the row which has used id =7

df6 = df5[df5.index == picked_userid].dropna(axis=1, how='all')
df6

In [ ]:
# prodcuts that similar users took. Removing products that none of the similar users have taken
similar_user_products = df5[df5.index.isin(similar_users.index)].dropna(axis=1, how='all')
similar_user_products

In [ ]:
# Remove the products taken from the products list
similar_user_products.drop(df6.columns,axis=1, inplace=True, errors='ignore')

# Take a look at the data
similar_user_products

In [ ]:
#now the recommendation part comes and it will mostly depend on the products taken by similar users

In [ ]:
# A dictionary to store product scores
product_score = {}

# Loop through products
for i in similar_user_products.columns:
  # Get the ratings for poduct i
  product_no = similar_user_products[i]
  # Create a variable to store the score
  total = 0
  # Create a variable to store the number of scores
  count = 0
  # Loop through similar users
  for u in similar_users.index:
    # If the movie has rating
    if pd.isna(product_no[u]) == False:
      # Score is the sum of user similarity score multiply by the product order numbers
      score = similar_users[u] * product_no[u]
      # Add the score to the total score for the product  so far
      total += score
      # Add 1 to the count
      count +=1
  # Get the average score for the product
  product_score[i] = total / count

# Convert dictionary to pandas dataframe
product_score = pd.DataFrame(product_score.items(), columns=['product name', 'product_score'])

# Sort the product by score
ranked_productscore = product_score.sort_values(by='product_score', ascending=False)

# Select top z products
z = 10
ranked_productscore.head(z)

it will finally give the dataframe that has top 5-10 recommended products for targeted user 7